In [1]:
import os 

In [2]:
%pwd

'c:\\development\\Machine-Learning\\end-to-end-ml-project\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\development\\Machine-Learning\\end-to-end-ml-project'

In [5]:
os.environ['MLFLOW_TRACKING_URI'] = 'https://dagshub.com/melihaltin/end-to-end-ml-project.mlflow'
os.environ['MLFLOW_TRACKING_USERNAME'] = 'melihaltin'
os.environ['MLFLOW_TRACKING_PASSWORD'] = '61b4d7966434a0202f289ca14a1157c66d2b487f'

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    X_test_data_path: Path
    y_test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_path: Path
    target_column: str
    mlflow_uri: str
    

In [7]:
from ml_project.constants import *
from ml_project.utils.common import read_yaml , create_directories, save_json

In [8]:
CONFIG_FILE_PATH

WindowsPath('config/config.yaml')

In [9]:
class ConfigurationManager:
    def __init__(self, config_filepath= CONFIG_FILE_PATH , params_filepath=PARAMS_FILE_PATH ,schema_filepath=SCHEMA_FILE_PATH ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
        
    def get_model_evaluation_config(self):
        config = self.config.model_evaluation
        params = self.params.LGBM
        schema = self.schema.TARGET
        
        create_directories([config.root_dir])
        
        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            X_test_data_path=Path(config.X_test_data_path),
            y_test_data_path=Path(config.y_test_data_path),
            model_path=Path(config.model_path),
            all_params=params,
            metric_file_path=Path(config.metric_file_path),
            target_column=schema.y,
            mlflow_uri=config.mlflow_uri
            
        )
        
        return model_evaluation_config
             

In [10]:
import pandas as pd 
import mlflow 
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, accuracy_score
from urllib.parse import urlparse
import mlflow.sklearn
import numpy as np
import joblib

[2024-02-27 10:23:58,067: INFO: utils:  Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.]
[2024-02-27 10:23:58,067: INFO: utils:  NumExpr defaulting to 8 threads.]


In [21]:
class ModelEvaluator:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
        
    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        acc = accuracy_score(actual, pred)
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2, acc
    
    def log_into_mlflow(self):
        mlflow.end_run()
        X_test = pd.read_csv(self.config.X_test_data_path)
        y_test = pd.read_csv(self.config.y_test_data_path)
        
        model = joblib.load(self.config.model_path)
      
        mlflow.set_registry_uri(self.config.mlflow_uri)

         
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        print(self.config.all_params)
        with mlflow.start_run():
            prediction_qualities = model.predict(X_test)
            (rmse, mae, r2 , acc) = self.eval_metrics(y_test, prediction_qualities)
            
            scores = {
                "acc": acc,
                "rmse": rmse,
                "mae": mae,
                "r2": r2
            }
            
            save_json(self.config.metric_file_path, data = scores)
            
            mlflow.log_params(self.config.all_params)
            
            
            mlflow.log_metrics(scores)
            # mlflow.log_metrics('mae', mae)
            # mlflow.log_metrics('r2', r2)
            
            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(model, "model" , registered_model_name="LGBM")
                
            else:
                mlflow.sklearn.log_model(model, "model")        
        
        

In [22]:
try:
    config = ConfigurationManager().get_model_evaluation_config()
    model_evaluator = ModelEvaluator(config)
    model_evaluator.log_into_mlflow()
except Exception as e:
    raise e 

[2024-02-27 10:35:19,911: INFO: common:  yaml file: config\config.yaml loaded successfully]
[2024-02-27 10:35:19,912: INFO: common:  yaml file: params.yaml loaded successfully]
[2024-02-27 10:35:19,914: INFO: common:  yaml file: schema.yaml loaded successfully]
[2024-02-27 10:35:19,915: INFO: common:  created directory at: artifacts]
[2024-02-27 10:35:19,916: INFO: common:  created directory at: artifacts/model_evaluation]


c:\Users\melih\anaconda3\envs\myenv\Lib\site-packages\sklearn\base.py:347: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.4.1.post1 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


{'alpha': 0.9, 'n_estimators': 200, 'learning_rate': 0.15, 'max_depth': 8}
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[2024-02-27 10:35:20,667: INFO: common:  json file saved at: artifacts\model_evaluation\metrics.json]


Registered model 'LGBM' already exists. Creating a new version of this model...
2024/02/27 10:35:27 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: LGBM, version 3
Created version '3' of model 'LGBM'.
